# Deploy to Azure Function App - serverless approach

Following [this tutorial](https://docs.microsoft.com/en-us/azure/azure-functions/create-first-function-cli-python?tabs=azure-cli%2Ccmd%2Cbrowser).

Steps:

1. Setup
2. Deploy model
3. Test model
4. Delete endpoint

Model to be deployed:

In [ ]:
model_path = "../../models/model.pkl"

In [ ]:
resource_group = "azure-cognitive-services-accelerator"

## 1. Setup
### 1.1 Setup Azure Functions CLI

Install Azure Functions CLI package following [the docs](https://learn.microsoft.com/en-us/azure/azure-functions/functions-run-local?tabs=v4%2Cmacos%2Ccsharp%2Cportal%2Cbash#install-the-azure-functions-core-tools) (you may need `xcode-select --install` too):

In [ ]:
!brew tap azure/functions

In [ ]:
!brew install azure-functions-core-tools@4

### 1.2 Setup location Azure Function

Initialise local function project folder:

In [ ]:
!func init sample_project --python

In [ ]:
%cd sample_project

Create new function that is triggered by HTTP:

In [ ]:
!func new --name sample_endpoint --template "HTTP trigger" --authlevel "anonymous" --worker-runtime python

Copy scikit-learn model and add inferencing script + dependencies. Make sure scikit-learn is same version in which model was created otherwise may get compatibility errors (e.g. see [here](https://github.com/scikit-learn/scikit-learn/issues/24387) for common scikit-learn error and see [here](https://github.com/scikit-image/scikit-image/issues/5060) for a python3.8 vs 3.9 issue for older sklearn versions)

In [ ]:
!cp $model_path sample_endpoint

In [ ]:
%%writefile -a requirements.txt

scikit-learn==1.2.0

In [ ]:
%%writefile sample_endpoint/__init__.py
import azure.functions as func
import numpy as np
import logging, sklearn, joblib, json

def main(req: func.HttpRequest) -> func.HttpResponse:
    logging.info(sklearn.__version__) # check this matches sklearn version in which model was trained
    pred = joblib.load('sample_endpoint/model.pkl').predict(np.array(req.get_json()['data']))
    return func.HttpResponse(json.dumps({"prediction": pred.tolist()}))

Optional: test model locally using `func start` in a new terminal and query local API with requests in Test section below.

### 1.3 Create resources: Azure Function App and Storage Account

Create storage account under appropriate resource group (ideally keep all assets under same resource group for cost tracking).

**Note** that storage account and function app can't include `_` in their names, and function app creation fails silently if you don't follow this.

In [ ]:
!az storage account create --name sampleprojectstorage --resource-group $resource_group --sku Standard_LRS

Create function app under same resource group and location. Check that runtime-version is the same as python version in which model was created/compatible with the sklearn version.

In [ ]:
!az functionapp create --resource-group $resource_group --consumption-plan-location uksouth --runtime python --runtime-version 3.10 --functions-version 4 --name sampleprojectapp --os-type linux --storage-account sampleprojectstorage

Use `!az functionapp update --resource-group $resource_group --name sample_project` to rerun the above command.

## 2. Deploy model

Publish function app (i.e. deploy). If first time results in Deploy Failed, just try rerunning the command (**shrugs shoulders**). Take note of the invoke URL at the end for API queries.

In [ ]:
!func azure functionapp publish sampleprojectapp

In [ ]:
endpoint_url = "https://sampleprojectapp.azurewebsites.net/api/sample_endpoint"

## 3. Test endpoint

You can manage and monitor the logs of the function app on the Azure portal under Function App.

In [ ]:
import requests
import json
import numpy as np

input_payload = json.dumps({
    'data': np.load("../../data/diabetes.npz")["X_test"][:2].tolist(),
})

requests.post(endpoint_url, input_payload, headers={'Content-Type':'application/json'}).json()

## 4. Delete endpoint

Delete created resources. Or you could do this in Azure portal.

In [ ]:
!az functionapp delete --name sampleprojectapp --resource-group $resource_group

In [ ]:
!az storage account delete --name sampleprojectstorage --resource-group $resource_group -y

Optional: delete local files

In [ ]:
%cd ..

In [ ]:
!rm -r sample_project